<a target="_blank" href="https://colab.research.google.com/github/https://colab.research.google.com/github/mcosarinsky/TP-NLP/blob/main/text_embeddings.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!git clone https://github.com/mcosarinsky/TP-NLP.git
!pip install faiss-cpu

Cloning into 'TP-NLP'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 7), reused 17 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 12.56 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (7/7), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

os.chdir("/content/TP-NLP")

df_articulos = pd.read_csv('articulos_subset.csv')
df_qa = pd.read_csv('qa_dataset.csv')

## Articles embeddings

In [3]:
from transformers import AutoModel
import faiss
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True)
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

configuration_xlm_roberta.py:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_lora.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- xlm_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- modeling_xlm_roberta.py
- embedding.py
- mlp.py
- mha.py
- block.py
- xlm_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/xlm-roberta-flash-implementation:
- modeling_lora.py
- modeling_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [4]:
dim = 1024
index = faiss.IndexIDMap(faiss.IndexFlatIP(dim))

for _, sample in tqdm(df_articulos.iterrows(), total=len(df_articulos)):
    id = np.array([sample['id']], dtype=np.int64)
    article = f"Diario:{sample['site']}\nTitulo:{sample['title']}\nFecha: {sample['published date']}\nDescripción: {sample['description']}\nContenido: {sample['text']}"

    embedding = model.encode(article, task='retrieval.passage', max_length=2048)
    embedding = embedding.reshape(1,-1)
    faiss.normalize_L2(embedding)
    index.add_with_ids(embedding, id)

faiss.write_index(index, "articles_index.index")

100%|██████████| 1000/1000 [08:26<00:00,  1.98it/s]


## Retrieval

In [10]:
index = faiss.read_index("articles_index.index")
k = 3

queries = df_qa.Pregunta.to_list()
queries_embeddings = model.encode(queries, task='retrieval.query')
faiss.normalize_L2(queries_embeddings)

distances, indices = index.search(queries_embeddings, k)

In [16]:
# Add retrieved articles id to dataframe
for i in range(k):
    df_qa[f'top_{i+1}_id'] = indices[:, i]

df_qa.to_csv('qa_dataset.csv', index=False)